<a href="https://colab.research.google.com/github/AbsolutUnit/Textract-Pipeline/blob/main/Textract_Conversion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Preliminary Pruning and Organization
Setting up the format of the data, as well as how it'll be processed before running AWS's textract.

We'll be importing boto3, which gives us access to AWS API's and tools.

In [ ]:
!pip install boto3
!pip install awscli

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os
import boto3
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from botocore import exceptions as ex

In [ ]:
!export AWS_SHARED_CREDENTIALS_FILE=/content/drive/My\ Drive/config/awscli.ini
path = "/content/drive/My Drive/config/awscli.ini"
os.environ['AWS_SHARED_CREDENTIALS_FILE'] = path
print(os.environ['AWS_SHARED_CREDENTIALS_FILE'])

/content/drive/My Drive/config/awscli.ini


In [ ]:
!aws s3 ls s3:// --recursive --human-readable --summarize

2021-07-14 19:36:35 bucketofpdfs
2021-07-14 20:37:35 outputjsonroseai

Total Objects: 0
   Total Size: 0 Bytes


Here we can initialize the global variables, like the bucket name and other flags to keep track of during the code execution.

In [ ]:
input_bucket_name = "bucketofpdfs"          # Bucket created earlier under anbarua@cs.stonybrook.edu
json_bucket_name = "outputjsonroseai"

Skeleton code for a function that takes the files and uploads to a bucket. This bucket currently acts as a repository of PDF files ordered by date added and labelled by file name.

**To do:** Filter out or deal with invalid file types, or corrupted files/large files that can't be segmented like below. 

In [ ]:
def pdf_up(doc_name):
  s3 = boto3.client('s3')
  try:
    retval = s3.upload_file(doc_name, input_bucket_name, doc_name)
  except ex.ClientError as i:
    print("Error: ", i)
    return False
  return True

This method, parse_file, chooses a specific file within the bucket to begin the Textract document text detection and extraction from. This returns the job ID, since it runs concurrent to the code via a client.

In [ ]:
def parse_file(bucket_name, doc_name):
  client = boto3.client('textract')
  retval = client.start_document_text_detection(
      DocumentLocation = {'S3Object' : {'Bucket': bucket_name, 'Name' : doc_name}})
  return retval["JobID"]

Skeleton code for main, allowing the entire py file to be easily run from CLI. An idea would be to have flags like -u or -c for upload or conversion. The bucket handling is all within the file, but an example execution could be:

python3 Textract-Conversion -u A.pdf B.pdf C.pdf

python3 Textract-Conversion -c A B C

or even the following

python3 Textract-Conversion -u A.pdf B.pdf C.pdf -c

In [ ]:
def main():
  # Take in parameters for easier scripting like -u for upload, -c to convert
  # pdf upload all valid file names if indicated to upload
  # Run compound method which includes parse and wait for JobID completion and add json to second bucket
  # return True # after all processes complete
  return False

if __name__ == "__main__":
  main()